In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import subprocess
from mango.tuner import Tuner
import numpy as np
import csv
import time
import random
import pickle
import itertools
import re

## General Settings

In [2]:
NAS_EPOCHS = 20 #NAS epochs
FlashMax = 250 #250 megabytes
log_file_name = 'log_NAS_Tracker.csv' #log file for NAS
if os.path.exists(log_file_name):
    os.remove(log_file_name)
row_write = ['score', 'mota','idf','flash','actual_num_stacks','kernel_size',
             'head_conv_value','activations','pre_thresh','track_thresh']
with open(log_file_name, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_write)
if os.path.exists(log_file_name[0:-4]+'.p'):
    os.remove(log_file_name[0:-4]+'.p')

In [3]:
class cd:
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)
    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)
    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [4]:
with cd('tracker/CenterTrack/src/lib/model/networks/DCNv2'):
    t = os.system('python setup.py build develop >/dev/null 2>&1')

FileNotFoundError: [Errno 2] No such file or directory: 'tracker/CenterTrack/src/lib/model/networks/DCNv2'

## NAS

In [5]:
def objective_NN(actual_num_stacks=1, kernel_size=1, 
                 head_conv_value=256,activations=True,
                pre_thresh=0.5,track_thresh=0.4, current_exp=0):
    print(actual_num_stacks,kernel_size,head_conv_value,activations,pre_thresh,track_thresh, current_exp)
    score = 0.0
    
    
    # Setting up model and immediately quitting:
    const_string = "python main.py tracking --exp_id mot17_half_exp" + str(current_exp) + \
        " --dataset mot --dataset_version 17halftrain --pre_hm --ltrb_amodal" + \
        " --same_aug --hm_disturb 0.05 --lost_disturb 0.4 --fp_disturb 0.1 --gpus 0,1 --num_epochs 70 --batch_size 16"
    variable_string = ' --actual_num_stacks '+str(actual_num_stacks)+' --kernel_size '+str(kernel_size)+ \
                  ' --head_conv_value '+str(head_conv_value)+' --activations '+str(activations)
    cmd = const_string + variable_string + " --no_train"  # DO NOT TRAIN
    with cd('updated_trackers/CenterTrack/src/'):
        os.system(cmd)
        
    # If the model exceeds a size threshold, we immediately write results and return
    with cd('updated_trackers/CenterTrack/models/'):
        flash = os.path.getsize('model_last.pth')/1e6 #megabytes
    if(flash >= FlashMax):
        score = -10000
        row_write = [score, -1,-1,flash,actual_num_stacks,
             kernel_size,head_conv_value,activations,pre_thresh,track_thresh]
        
        print('Design choice:',row_write)
        with open(log_file_name, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(row_write)
            
        return score
        
    
    # We continue training
    print("Starting training...")
    cmd = const_string + variable_string  + ' > debug.txt'
    with cd('updated_trackers/CenterTrack/src/'):
        os.system(cmd)
        
        
    # Set up testing    
    const_string_2 = "python test.py tracking --exp_id mot17_half_exp" + str(current_exp) + \
        " --dataset mot --dataset_version 17halfval --pre_hm --ltrb_amodal --resume"
    variable_string_2 = ' --pre_thresh '+str(pre_thresh)+' --track_thresh '+str(track_thresh)
    cmd2 = const_string_2 + variable_string_2 + ' > results_tracker.txt'
    
    # Run testing
    with cd('updated_trackers/CenterTrack/src/'):
        os.system(cmd2)
        
    # Analyze results
    with cd('updated_trackers/CenterTrack/src/'):
        with open('results_tracker.txt') as f:
            z = f.readlines()
        f.close()  
    results = []
    for x in z:
        if "OVERALL" in x:
            results.append(x)
    results = results[-1]
    mota = float(re.split('\s+', results)[14][0:-1])
    idf =  float(re.split('\s+', results)[1][0:-1])
    with cd('updated_trackers/CenterTrack/models/'):
        flash = os.path.getsize('model_last.pth')/1e6 #megabytes
    
    if(flash < FlashMax):
        score = mota + idf - flash
    else:
        score = -10000
        
    row_write = [score, mota,idf,flash,actual_num_stacks,
             kernel_size,head_conv_value,activations,pre_thresh,track_thresh]
    print('Design choice:',row_write)
    with open(log_file_name, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row_write)   
        
    return score

In [6]:
def save_res(data, file_name):
    pickle.dump( data, open( file_name, "wb" ) )
    
param_dict = {
    'actual_num_stacks': range(1,5),
    'kernel_size': [1,3,5,7,9,11,13,15,17,19,21,23], # # kernel_size = [x for x in range(1,24,2)]
    'head_conv_value': [50,100,150,200,250,300],
    'activations': [True, False],
    'pre_thresh': np.linspace(0.1,0.9,9),
    'track_thresh': np.linspace(0.1,0.9,9)
}

def objfunc(args_list):
    global CURRENT_EXPERIMENT
    
    print("Current EXP: " + str(CURRENT_EXPERIMENT))
    
    objective_evaluated = []
    
    start_time = time.time()
    
    for current_iter, hyper_par in enumerate(args_list):
        
        actual_num_stacks = hyper_par['actual_num_stacks']
        kernel_size = hyper_par['kernel_size']
        head_conv_value = hyper_par['head_conv_value']
        activations = hyper_par['activations']
        pre_thresh = hyper_par['pre_thresh']
        track_thresh = hyper_par['track_thresh']
            
        objective = objective_NN(actual_num_stacks=actual_num_stacks,
                                 kernel_size=kernel_size,head_conv_value=head_conv_value,
                                 activations=activations, pre_thresh = pre_thresh, 
                                 track_thresh = track_thresh, current_exp=CURRENT_EXPERIMENT)
        objective_evaluated.append(objective)
        
        end_time = time.time()
        print('objective:', objective, ' time:',end_time-start_time)
        CURRENT_EXPERIMENT += 1
        
    return objective_evaluated

In [7]:
CURRENT_EXPERIMENT = 0
conf_Dict = dict()
conf_Dict['batch_size'] = 1 
conf_Dict['num_iteration'] = NAS_EPOCHS
conf_Dict['initial_random']= 5
tuner = Tuner(param_dict, objfunc,conf_Dict)
all_runs = []
results = tuner.maximize()
all_runs.append(results)
save_res(all_runs,log_file_name[0:-4]+'.p')

Current EXP: 0
2 15 250 True 0.7000000000000001 0.6 0
Using tensorboardX
Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [8, 8]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, activations='True', actual_num_stacks='2', add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=16, bird_view_world_size=64, chunk_sizes=[8, 8], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/home/brianw/Documents/PTZCameraRecognition/brian_stuff/ByteTrack/updated_trackers/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halftrain', debug=0, debug_dir='/home/brianw/Documents/PTZCameraRecognition/brian_stuff/ByteTrack/updated_trackers/CenterTrack/src/lib/../../exp

Design choice: [-10000, -1, -1, 274.382481, 3, 7, 300, True, 0.5, 0.1]
objective: -10000  time: 7.036720514297485
1 7 300 True 0.7000000000000001 0.9 2
Using tensorboardX
Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [8, 8]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, activations='True', actual_num_stacks='1', add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=16, bird_view_world_size=64, chunk_sizes=[8, 8], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/home/brianw/Documents/PTZCameraRecognition/brian_stuff/ByteTrack/updated_trackers/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halftrain', debug=0, debug_dir='/home/brianw/D

/home/brianw/Documents/PTZCameraRecognition/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/brianw/Documents/PTZCameraRecognition/venv/lib/python3.8/site-packages/sklearn/utils/linear_assignment_.py:18: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
02:56:15 INFO - Found 7 groundtruths and 7 test files.
02:56:15 INFO - Available LAP solvers ['lap', 'scipy']
02:56:15 INFO - Default LAP solver 'lap'
02:56:15 INFO - Loading files.
02:56:16 INFO - Comparing MOT17-11-FRCNN...
02:56:16 INFO - Comparing MOT17-04-FRCNN...
02:56:17 INFO - Comparing MOT17-05-FRCNN...
02:56:17 INFO - Comparing MOT17-09-FRCNN...
02:56:17 INFO - Comparing MOT17-02-FRCNN...
02:56

Design choice: [-10000, 27.8, 52.2, 293.605077, 1, 7, 300, True, 0.7000000000000001, 0.9]
objective: -10000  time: 12985.103556156158
3 1 250 True 0.2 0.4 3
Using tensorboardX
Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [8, 8]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, activations='True', actual_num_stacks='3', add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=16, bird_view_world_size=64, chunk_sizes=[8, 8], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/home/brianw/Documents/PTZCameraRecognition/brian_stuff/ByteTrack/updated_trackers/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halftrain', debug=0, debug_dir='/home/bri

/home/brianw/Documents/PTZCameraRecognition/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/brianw/Documents/PTZCameraRecognition/venv/lib/python3.8/site-packages/sklearn/utils/linear_assignment_.py:18: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
05:48:51 INFO - Found 7 groundtruths and 7 test files.
05:48:51 INFO - Available LAP solvers ['lap', 'scipy']
05:48:51 INFO - Default LAP solver 'lap'
05:48:51 INFO - Loading files.
05:48:52 INFO - Comparing MOT17-11-FRCNN...
05:48:53 INFO - Comparing MOT17-04-FRCNN...
05:48:53 INFO - Comparing MOT17-05-FRCNN...
05:48:54 INFO - Comparing MOT17-09-FRCNN...
05:48:54 INFO - Comparing MOT17-02-FRCNN...
05:48

...

Design choice: [-10000, 31.5, 53.4, 261.242013, 3, 7, 50, True, 0.9, 0.5]
objective: -10000  time: 10684.362088441849
Current EXP: 22
2 1 250 True 0.2 0.2 22
Using tensorboardX
Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [8, 8]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, activations='True', actual_num_stacks='2', add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=16, bird_view_world_size=64, chunk_sizes=[8, 8], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/home/brianw/Documents/PTZCameraRecognition/brian_stuff/ByteTrack/updated_trackers/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halftrain', debug=0, debug_dir='/home/br

/home/brianw/Documents/PTZCameraRecognition/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/brianw/Documents/PTZCameraRecognition/venv/lib/python3.8/site-packages/sklearn/utils/linear_assignment_.py:18: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
12:30:38 INFO - Found 7 groundtruths and 7 test files.
12:30:38 INFO - Available LAP solvers ['lap', 'scipy']
12:30:38 INFO - Default LAP solver 'lap'
12:30:38 INFO - Loading files.
12:30:39 INFO - Comparing MOT17-11-FRCNN...
12:30:39 INFO - Comparing MOT17-04-FRCNN...
12:30:40 INFO - Comparing MOT17-05-FRCNN...
12:30:40 INFO - Comparing MOT17-09-FRCNN...
12:30:40 INFO - Comparing MOT17-02-FRCNN...
12:30

Design choice: [-163.281113, 25.8, 52.8, 241.881113, 2, 1, 250, True, 0.2, 0.2]
objective: -163.281113  time: 9472.3073720932
Current EXP: 23
1 9 100 False 0.7000000000000001 0.5 23
Using tensorboardX
Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [8, 8]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, activations='False', actual_num_stacks='1', add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=16, bird_view_world_size=64, chunk_sizes=[8, 8], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/home/brianw/Documents/PTZCameraRecognition/brian_stuff/ByteTrack/updated_trackers/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halftrain', deb

/home/brianw/Documents/PTZCameraRecognition/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/brianw/Documents/PTZCameraRecognition/venv/lib/python3.8/site-packages/sklearn/utils/linear_assignment_.py:18: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
03:14:03 INFO - Found 7 groundtruths and 7 test files.
03:14:03 INFO - Available LAP solvers ['lap', 'scipy']
03:14:03 INFO - Default LAP solver 'lap'
03:14:03 INFO - Loading files.
03:14:04 INFO - Comparing MOT17-11-FRCNN...
03:14:05 INFO - Comparing MOT17-04-FRCNN...
03:14:05 INFO - Comparing MOT17-05-FRCNN...
03:14:05 INFO - Comparing MOT17-09-FRCNN...
03:14:06 INFO - Comparing MOT17-02-FRCNN...
03:14

Design choice: [-10000, 36.1, 54.6, 268.222997, 1, 9, 100, False, 0.7000000000000001, 0.5]
objective: -10000  time: 9804.459024429321
Current EXP: 24
2 9 300 True 0.1 0.4 24
Using tensorboardX
Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [8, 8]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, activations='True', actual_num_stacks='2', add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=16, bird_view_world_size=64, chunk_sizes=[8, 8], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/home/brianw/Documents/PTZCameraRecognition/brian_stuff/ByteTrack/updated_trackers/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halftrain', debug=0, deb